<a href="https://colab.research.google.com/github/monimiller/womens_wc_23/blob/main/run_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.6 MB/s eta 0:00:00


In [2]:
# Basics
import pandas as pd
import numpy as np
# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#Catboost
from catboost import CatBoostClassifier, CatBoostRegressor

In [3]:
!wget https://github.com/monimiller/womens_wc_23/raw/main/models/catboost_model_20230717_2123.cbm

--2023-07-20 01:56:51--  https://github.com/monimiller/womens_wc_23/raw/main/models/catboost_model_20230717_2123.cbm
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/monimiller/womens_wc_23/main/models/catboost_model_20230717_2123.cbm [following]
--2023-07-20 01:56:51--  https://raw.githubusercontent.com/monimiller/womens_wc_23/main/models/catboost_model_20230717_2123.cbm
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21314844 (20M) [application/octet-stream]
Saving to: ‘catboost_model_20230717_2123.cbm’

catboost_model_2023 100%[===================>]  20.33M   128MB/s    in 0.2s    

2023-07-20

In [4]:
model = CatBoostClassifier().load_model("catboost_model_20230717_2123.cbm")
teams_performance = pd.read_parquet("team_performance_20230717.parquet.gzip")

In [8]:
group_a = pd.DataFrame({
                      'home_team': ['New Zealand', 'Philippines', 'New Zealand', 'Switzerland', 'Switzerland', 'Norway'],
                      'away_team': ['Norway', 'Switzerland', 'Philippines', 'Norway', 'New Zealand', 'Philippines']
                      })

group_b = pd.DataFrame({
                      'home_team': ['Australia', 'Nigeria', 'Canada', 'Australia', 'Republic of Ireland', 'Canada'],
                      'away_team': ['Republic of Ireland', 'Canada', 'Republic of Ireland', 'Nigeria', 'Nigeria', 'Australia']
                      })

group_c = pd.DataFrame({
                      'home_team': ['Spain', 'Zambia', 'Japan', 'Spain', 'Costa Rica', 'Japan'],
                      'away_team': ['Costa Rica', 'Japan', 'Costa Rica', 'Zambia', 'Zambia', 'Spain']
                      })

group_d = pd.DataFrame({
                      'home_team': ['England', 'Denmark', 'England', 'China PR', 'Haiti', 'China PR'],
                      'away_team': ['Haiti', 'China PR', 'Denmark', 'Haiti', 'Denmark', 'England']
                      })

group_e = pd.DataFrame({
                      'home_team': ['United States', 'Netherlands', 'United States', 'Portugal', 'Portugal', 'Vietnam'],
                      'away_team': ['Vietnam', 'Portugal', 'Netherlands', 'Vietnam', 'United States', 'Netherlands']
                      })

group_f = pd.DataFrame({
                      'home_team': ['France', 'Brazil', 'France', 'Panama', 'Panama', 'Jamaica'],
                      'away_team': ['Jamaica', 'Panama', 'Brazil', 'Jamaica', 'France', 'Brazil']
                      })

group_g = pd.DataFrame({
                      'home_team': ['Sweden', 'Italy', 'Argentina', 'Sweden', 'South Africa', 'Argentina'],
                      'away_team': ['South Africa', 'Argentina', 'South Africa', 'Italy', 'Italy', 'Sweden']
                      })

group_h = pd.DataFrame({
                      'home_team': ['Germany', 'Columbia', 'South Korea', 'Germany', 'South Korea', 'Morocco'],
                      'away_team': ['Morocco', 'South Korea', 'Morocco', 'Columbia', 'Germany', 'Columbia']
                      })

In [6]:
def encode_variables(df):
  "Input a dataframe and return a dataframe whith categorical variables encoded numericaly"
  # Find categorical vars
  cat_vars = df.select_dtypes(include=['object', 'category']).columns
  # Loop and transform
  for variable in cat_vars:
    df[variable] = df[variable].factorize()[0]

  return df

In [9]:
groups = [group_a, group_b, group_c, group_d, group_e, group_f, group_g, group_h]

def make_predictions(group_df):
    X_input = (
    group_df
    .merge(teams_performance, left_on= 'home_team', right_on='team', how='left').fillna(0)
    .merge(teams_performance, left_on= 'away_team', right_on='team', suffixes=['_home','_away'], how='left').fillna(0) )
    X_input.drop(['team_home', 'team_away'], axis=1, inplace=True)

    X_input = encode_variables(X_input)
    # Results with CatBoost
    preds = model.predict_proba(X_input)
    # Predictions of the results
    group_df['away'] = pd.DataFrame(preds)[0]
    group_df['draw'] = pd.DataFrame(preds)[1]
    group_df['home'] = pd.DataFrame(preds)[2]

    print(group_df)
    print('\n')

for group in groups:
    make_predictions(group)

     home_team    away_team      away      draw      home
0  New Zealand       Norway  0.815434  0.167203  0.017363
1  Philippines  Switzerland  0.060202  0.008959  0.930839
2  New Zealand  Philippines  0.000416  0.013989  0.985596
3  Switzerland       Norway  0.816193  0.181277  0.002531
4  Switzerland  New Zealand  0.230713  0.582597  0.186690
5       Norway  Philippines  0.000083  0.003750  0.996167


             home_team            away_team          away      draw      home
0            Australia  Republic of Ireland  4.376515e-07  0.001193  0.998806
1              Nigeria               Canada  9.801622e-01  0.014929  0.004909
2               Canada  Republic of Ireland  4.408379e-06  0.000599  0.999396
3            Australia              Nigeria  1.319833e-05  0.001892  0.998095
4  Republic of Ireland              Nigeria  9.734720e-04  0.002001  0.997025
5               Canada            Australia  1.628516e-04  0.000845  0.998993


    home_team   away_team      away      dra